In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab\ Notebooks/NLP/HW2/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/NLP/HW2


# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [2]:
! pip install seaborn
! pip install opencc
! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 85.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [3]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,src,tgt
0,14*(43+20)=,882
1,(6+1)*5=,35
2,13+32+29=,74
3,31*(3-11)=,-248
4,24*49+1=,1177


In [4]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [5]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite

all_chars = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "+",
    "-",
    "*",
    "(",
    ")",
    "=",
]
special_tokens = ["<pad>", "<eos>"]
all_tokens = special_tokens + all_chars

char_to_id = {char: idx for idx, char in enumerate(all_tokens)}
id_to_char = {idx: char for idx, char in enumerate(all_tokens)}

vocab_size = len(char_to_id)
print('Vocab size{}'.format(vocab_size))

Vocab size18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [6]:
# Write your code here

def data_preprocess(df: pd.DataFrame, char_to_id: dict) -> pd.DataFrame:
    df = df.copy()
    char_id_list = []
    label_id_list = []
    src_len_list = []

    for sent in df["src"]:
        sent_len = len(sent)
        sent = sent.split("=")
        sent_train = sent[0]
        sent_tgt = sent[1]

        char_id = []
        label_id = []

        # Input part before '='
        for char in sent_train:
            char_id.append(char_to_id[char])
            label_id.append(char_to_id["<pad>"])

        # '=' symbol
        char_id.append(char_to_id["="])
        label_id.append(char_to_id[sent_tgt[0]])

        # Answer part - each position predicts NEXT character
        for i, char in enumerate(sent_tgt):
            char_id.append(char_to_id[char])
            if i < len(sent_tgt) - 1:
                label_id.append(char_to_id[sent_tgt[i + 1]])
            else:
                label_id.append(char_to_id["<eos>"])

        #   0     +     0   =   0   <eos>
        # <pad> <pad> <pad> 0 <eos> <pad>

        # End of sequence token
        char_id.append(char_to_id["<eos>"])
        label_id.append(char_to_id["<pad>"])

        src_len_list.append(sent_len)
        char_id_list.append(char_id)
        label_id_list.append(label_id)

    df["len"] = src_len_list
    df["char_id_list"] = char_id_list
    df["label_id_list"] = label_id_list

    return df


df_train = data_preprocess(df_train, char_to_id)

df_train.head()

,src,tgt,len,char_id_list,label_id_list
0,14*(43+20)=882,882,14,"[3, 6, 14, 15, 6, 5, 12, 4, 2, 16, 17, 10, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 4, 1, 0]"
1,(6+1)*5=35,35,10,"[15, 8, 12, 3, 16, 14, 7, 17, 5, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 5, 7, 1, 0]"
2,13+32+29=74,74,11,"[3, 5, 12, 5, 4, 12, 4, 11, 17, 9, 6, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 9, 6, 1, 0]"
3,31*(3-11)=-248,-248,14,"[5, 3, 14, 15, 5, 13, 3, 3, 16, 17, 13, 4, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 4, 6, 10, 1, 0]"
4,24*49+1=1177,1177,12,"[4, 6, 14, 6, 11, 12, 3, 17, 3, 3, 9, 9, 1]","[0, 0, 0, 0, 0, 0, 0, 3, 3, 9, 9, 1, 0]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [7]:
batch_size = 256
epochs = 5
embed_dim = 256
hidden_dim = 256
lr = 0.0011554
weight_decay = 0.0057802
momentum = 0.98805
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        # return the amount of data
        # return # Write your code here

        return len(self.sequences)

    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        # x = # Write your code here
        # y = # Write your code here

        x = self.sequences.iloc[index]["char_id_list"]
        y = self.sequences.iloc[index]["label_id_list"]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])

    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [9]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])

In [10]:
# Build dataloader of train set and eval set, collate_fn is the collate function
# dl_train = # Write your code here

dl_train = torch.utils.data.DataLoader(
    ds_train,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=os.cpu_count(),
    pin_memory=True,
    persistent_workers=True,
)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [11]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()

        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])

        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))

    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)

    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)

        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)

        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)

        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)

        batch_x = self.linear(batch_x)

        return batch_x

    def generator(self, start_char, max_len=200):

        char_list = [char_to_id[c] for c in start_char]

        next_char = None

        while len(char_list) < max_len:
            # Write your code here
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively
            # y = # Obtain the next token prediction y

            batch_x = torch.tensor(char_list).unsqueeze(0).to(device)
            batch_x_len = torch.tensor([len(char_list)])
            y = self.forward(batch_x, batch_x_len)
            logits = y[0, -1]

            # next_char = # Use argmax function to get the next token prediction

            next_char = torch.argmax(logits, dim=-1).item()

            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)

        return [id_to_char[ch_id] for ch_id in char_list]

In [12]:
torch.manual_seed(2)


# device = # Write your code here. Specify a device (cuda or cpu)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

In [13]:
# criterion = # Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
# optimizer = # Write your code here. Use Adam or AdamW for Optimizer

criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id["<pad>"])


! pip install git+https://github.com/KellerJordan/Muon
from muon import SingleDeviceMuon

muon_params = [
    p
    for layer in [model.rnn_layer1, model.rnn_layer2]
    for p in layer.parameters()
    if p.ndim >= 2
]

adamw_params = [
    *model.embedding.parameters(),
    *[
        p
        for layer in [model.rnn_layer1, model.rnn_layer2]
        for p in layer.parameters()
        if p.ndim < 2
    ],
    *model.linear.parameters(),
]

optimizers = [
    SingleDeviceMuon(muon_params, lr=lr, momentum=momentum),
    torch.optim.AdamW(adamw_params, lr=lr, weight_decay=weight_decay),
]

  Cloning https://github.com/KellerJordan/Muon to /tmp/pip-req-build-gu93hn79
  Running command git clone --filter=blob:none --quiet https://github.com/KellerJordan/Muon /tmp/pip-req-build-gu93hn79
  Resolved https://github.com/KellerJordan/Muon to commit f90a42b28e00b8d9d2d05865fe90d9f39abcbcbd
  Preparing metadata (setup.py) ... done
  Created wheel for muon-optimizer: filename=muon_optimizer-0.1.0-py3-none-any.whl size=7144 sha256=88207f602b23918af47536aaa2ae4f3d7f0386de67a29ebb31d379269359146a
  Stored in directory: /tmp/pip-ephem-wheel-cache-g2nsu2mh/wheels/6e/33/94/64d18603ba0f39064aab523d6edf493c388cfb7419bb5c9043
Successfully built muon-optimizer


# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [14]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient

        batch_x = batch_x.to(device, non_blocking=True)
        batch_y = batch_y.to(device, non_blocking=True)
        for optimizer in optimizers:
            optimizer.zero_grad()

        batch_pred_y = model(batch_x.to(device), batch_x_lens)

        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation

        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.view(-1))
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model

        for optimizer in optimizers:
            optimizer.step()

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())

    # Evaluate your model
    matched = 0
    total = 0
    bar_eval = tqdm(df_eval.iterrows(), desc=f"Validation epoch {epoch}")
    for _, row in bar_eval:
        batch_x = row['src']
        batch_y = row['tgt']

        # prediction = # An example of using generator: model.generator('1+1=')

        # Write your code here. Input the batch_x to the model and generate the predictions
        # Write your code here.
        # Check whether the prediction match the ground truths
        # Compute exact match (EM) on the eval dataset
        # EM = correct/total

        prediction = "".join(model.generator(batch_x, max_len=50))
        prediction = prediction.split("=")[-1].replace("<eos>", "")
        is_correct = int(prediction == batch_y)
        matched += is_correct
        total += 1

    print(matched/total)

Train epoch 1: 100%|██████████| 9255/9255 [05:12<00:00, 29.65it/s, loss=0.324]
Validation epoch 1: 263250it [34:35, 126.82it/s]


0.6449344729344729


Train epoch 2: 100%|██████████| 9255/9255 [04:52<00:00, 31.69it/s, loss=0.184]
Validation epoch 2: 263250it [32:23, 135.47it/s]


0.750579297245964


Train epoch 3: 100%|██████████| 9255/9255 [04:52<00:00, 31.62it/s, loss=0.129]
Validation epoch 3: 263250it [33:37, 130.50it/s]


0.846343779677113


Train epoch 4: 100%|██████████| 9255/9255 [04:57<00:00, 31.16it/s, loss=0.0765]
Validation epoch 4: 263250it [33:47, 129.84it/s]


0.9031225071225071


Train epoch 5: 100%|██████████| 9255/9255 [04:51<00:00, 31.76it/s, loss=0.0656]
Validation epoch 5: 263250it [31:49, 137.84it/s]

0.9299259259259259
